In [6]:
import tensorflow as tf
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, Input, GlobalAveragePooling2D
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from keras.optimizers import RMSprop, Adam
from keras import backend as K

In [7]:
height = 299
width = 299
batch_size = 2

train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False,
    validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    'D:\.Downloads\high_res\data',#'/content/drive/My Drive/malariaNN/high_res/data',
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')
validation_generator = train_datagen.flow_from_directory(
    'D:\.Downloads\high_res\data',#'/content/drive/My Drive/malariaNN/high_res/data',
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')

epochs = 20
steps = 170
val_steps = 40

Found 170 images belonging to 4 classes.
Found 40 images belonging to 4 classes.


In [10]:
input_shape = None

if K.image_data_format() == 'channels_first':
    input_shape = (3, width, height)
else:
    input_shape = (width, height, 3)

base_model = InceptionV3(weights='D:\.Downloads\models\largemodelv4\modelweights.h5', include_top=False, input_tensor=Input(shape=input_shape))

out = base_model.output
out = GlobalAveragePooling2D()(out)
out = Dense(1024, activation='relu')(out)
predictions = Dense(4, activation='softmax')(out)

model = Model(inputs=base_model.input, outputs=predictions)

# freeze all weights in imagenet trained base model, but not the extra buffer layer and output layer
for layer in base_model.layers:
    layer.trainable = False

LR = 0.007
RMS = Adam(lr=LR)
model.compile(optimizer=RMS, loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
history = model.fit(
    train_generator,
    epochs=epochs,
    steps_per_epoch=steps // batch_size,
    validation_data=validation_generator,
    validation_steps=val_steps // batch_size
)

Epoch 1/20
85/85 [==============================] - 26s 309ms/step - loss: 1.6543 - accuracy: 0.4882 - val_loss: 1.3845 - val_accuracy: 0.4500
Epoch 2/20
85/85 [==============================] - 23s 268ms/step - loss: 1.2106 - accuracy: 0.5529 - val_loss: 1.3735 - val_accuracy: 0.4250
Epoch 3/20
85/85 [==============================] - 19s 220ms/step - loss: 1.1742 - accuracy: 0.5294 - val_loss: 1.5570 - val_accuracy: 0.2000
Epoch 4/20
85/85 [==============================] - 20s 236ms/step - loss: 1.0944 - accuracy: 0.5294 - val_loss: 1.3804 - val_accuracy: 0.3000
Epoch 5/20
85/85 [==============================] - 18s 212ms/step - loss: 1.1924 - accuracy: 0.5471 - val_loss: 1.4935 - val_accuracy: 0.2750
Epoch 6/20
85/85 [==============================] - 18s 216ms/step - loss: 1.0897 - accuracy: 0.5529 - val_loss: 1.4229 - val_accuracy: 0.3250
Epoch 7/20
85/85 [==============================] - 18s 212ms/step - loss: 1.0145 - accuracy: 0.6176 - val_loss: 1.3212 - val_accuracy: 0.5000

In [12]:
accs = 0
for i in range(10):
    accuracy = model.evaluate(
        validation_generator,
        steps=val_steps // batch_size,
    )
    accs += accuracy[1]
print("Avg accuracy: " + str(accs/10))

20/20 [==============================] - 3s 163ms/step - loss: 1.2613 - accuracy: 0.5750
Avg accuracy: 0.5074999898672103
